# Linear Regression

The aim of this tutorial is to develop a model of linear regression. This is a fundamental algorithm in machine learning, and one that we will build upon later on in this course.

### Introduction

Say we have a collection of labelled examples $\{(\mathbf{x}_i, y_i)\}_{i=1}^N$, where $N$ is the size of the dataset, $\mathbf{x}_i$ is the feature vector of sample $i$ and $y_i$ is the label of sample $i$. Linear regression aims to fit a hyperplane to the data. A hyperplane is a generalisation of a plane in high dimensions. It is impossible to visualise a hyperplane, but the maths works just fine, so bear with me!

Based on this description, is linear regression a form of supervised learning, or unsupervised learning? Run the cell below to answer:
<span style="display:none" id="q1">W3sicXVlc3Rpb24iOiAiTGluZWFyIHJlZ3Jlc3Npb24gaXMgYSB0eXBlIG9mOiIsICJ0eXBlIjogIm11bHRpcGxlX2Nob2ljZSIsICJhbnN3ZXJzIjogW3siYW5zd2VyIjogIlN1cGVydmlzZWQgbGVhcm5pbmcuIiwgImNvcnJlY3QiOiB0cnVlLCAiZmVlZGJhY2siOiAiVGhhdCdzIHJpZ2h0LiBCZWNhdXNlIHRoZSBleGFtcGxlcyBhcmUgbGFiZWxsZWQsIHRoaXMgaXMgc3VwZXJ2aXNlZCBsZWFybmluZy4ifSwgeyJhbnN3ZXIiOiAiVW5zdXBlcnZpc2VkIGxlYXJuaW5nIiwgImNvcnJlY3QiOiBmYWxzZSwgImZlZWRiYWNrIjogIk5vLiBCZWNhdXNlIHRoZSBleGFtcGxlcyBhcmUgbGFiZWxsZWQsIHRoaXMgY2Fubm90IGJlIHVuc3VwZXJ2aXNlZCBsZWFybmluZy4ifV19XQ==</span> 

In [ ]:
from jupyterquiz import display_quiz
display_quiz("#q1")

Let us define the length of the feature vector, $\mathbf{x}_i$ (for all $i$) as $D$. If $D=1$, the model will fit a line to the data (a line of best fit). If $D=2$, the model will aim to fit a plane to the data. For $D>2$, it'll be a hyperplane that is fit to the data. 

In general terms, the model has the following form:

$$f_{\mathbf{w},b} = \mathbf{w} \mathbf{x} + b$$

Once the model is trained (that is appropriate values of $\mathbf{w}$ and $b$ have been found), it can be used for predictions at new values of $\mathbf{x}$. 

### Engineering example

To provide a concrete engineering example, we will consider the heat transfer through a single-glazed window. A window manufacturer may be interested in training a model that can predict the rate of heat transfer, $\dot{Q}$, for a given glass thickness, $L$, window area, $A$, and a given temperature difference across the glass, $\Delta T$, (the difference between the inside and outside temperature). 

In this problem, which variables are features, and which is the label? Run the cell below to answer:

<span style="display:none" id="q2">W3sicXVlc3Rpb24iOiAiV2hpY2ggb2YgdGhlIGZvbGxvd2luZyBhcmUgZmVhdHVyZXM6IiwgInR5cGUiOiAibWFueV9jaG9pY2UiLCAiYW5zd2VycyI6IFt7ImFuc3dlciI6ICJUaGUgdGhpY2tuZXNzIG9mIHRoZSBnbGFzcywgJEwkLiIsICJjb3JyZWN0IjogdHJ1ZX0sIHsiYW5zd2VyIjogIlRoZSB0ZW1wZXJhdHVyZSBkaWZmZXJlbmNlICRcXERlbHRhIFQkLiIsICJjb3JyZWN0IjogdHJ1ZX0sIHsiYW5zd2VyIjogIlRoZSB3aW5kb3cgYXJlYSwgJEEkLiIsICJjb3JyZWN0IjogdHJ1ZX0sIHsiYW5zd2VyIjogIlRoZSByYXRlIG9mIGhlYXQgdHJhbnNmZXIsICRcXGRvdHtRfSQuIiwgImNvcnJlY3QiOiBmYWxzZSwgImZlZWRiYWNrIjogIk5vLiBUaGlzIGlzIHRoZSBsYWJlbCJ9XX1d</span> 

In [ ]:
from jupyterquiz import display_quiz
display_quiz("#q2")

You may notice the features are the _independent_ variables of the problem, and the label is the _dependent_ variable. Features and labels is the terminology commonly used in machine learning, but there is no appreciable difference between features and independent variables, or between labels and dependent variables. 

To keep the analysis simple, let's  consider the case where the window thickness and Area are fixed. The manufacturer is now interested in a model that can predict the rate of heat loss for a given temperature difference. Because $A$ and $L$ are fixed, this model will only work for the specific window dimension that the model is trained for. 

We have some data in the file "window_heat.csv". You can see this file in the file explorer (usually to the left). We can also load it in to this notebook by running the cell below:

In [ ]:
import pandas as pd                 #Pandas is a library in Python we can use for loading csv files (amongst other things)
df = pd.read_csv('window_heat.csv') #here, we load the csv file into a Pandas dataframe object called 'df'
df                                  #this line causes the notebook to print out the dataframe so we can see its contents below:

The dataframe contains a column of temperature differences and the corresponding measured rate of heat transfer through the window. 
Try answering the following:

<span style="display:none" id="q3">W3sicXVlc3Rpb24iOiAiSW4gdGhpcyBleGFtcGxlLCB3aGljaCBvZiB0aGUgZm9sbG93aW5nIGFyZSB0cnVlOiIsICJ0eXBlIjogIm1hbnlfY2hvaWNlIiwgImFuc3dlcnMiOiBbeyJhbnN3ZXIiOiAiRD0xLiIsICJjb3JyZWN0IjogdHJ1ZSwgImZlZWRiYWNrIjogIlRoYXQncyByaWdodC4ifSwgeyJhbnN3ZXIiOiAiRD0yLiIsICJjb3JyZWN0IjogZmFsc2UsICJmZWVkYmFjayI6ICJObywgdGhlIGxlbmd0aCBvZiB0aGUgZmVhdHVyZSB2ZWN0b3IgaXMgMSAodGhlIHRlbXBlcmF0dXJlIGRpZmZlcmVuY2UgaXMgdGhlIG9ubHkgZmVhdHVyZSkuICRcXGRvdHtRfSQgaXMgdGhlIGxhYmVsLiJ9LCB7ImFuc3dlciI6ICJOPTEiLCAiY29ycmVjdCI6IGZhbHNlLCAiZmVlZGJhY2siOiAiTm8uIFRoZSBudW1iZXIgb2YgZGF0YSBzYW1wbGVzIGlzIGdyZWF0ZXIgdGhhbiBvbmUuIn0sIHsiYW5zd2VyIjogIk49MjQiLCAiY29ycmVjdCI6IHRydWUsICJmZWVkYmFjayI6ICJZdXAsIHRoZXJlIGFyZSAyNCBkYXRhIHNhbXBsZXMgb3Igb2JzZXJ2YXRpb25zLiJ9LCB7ImFuc3dlciI6ICJOPTIzIiwgImNvcnJlY3QiOiBmYWxzZSwgImZlZWRiYWNrIjogIk5vdCBxdWl0ZSwgdGhlcmUgYXJlIDI0IGRhdGEgc2FtcGxlcyBvciBvYnNlcnZhdGlvbnMgYmVjYXVzZSB0aGUgbGlzdCBzdGFydHMgZnJvbSAwLiJ9XX1d</span> 

In [ ]:
from jupyterquiz import display_quiz
display_quiz("#q3")

 We can plot the data as follows:

In [ ]:
import matplotlib.pyplot as plt         #matplotlib is a plotting library in Python
df.plot.scatter(x='dT[C]', y='Qdot[W]')       #We plot the dataframe as a scatter plot to visualise the output

As expected, the rate of heat loss in Watts is higher if the temperature difference between the inside and outside is greater. We can visually see the relationship is broadly a linear one. Our aim here is to fit a line to this data through linear regression.

First, we can get the raw data in numpy arrays to allow simple manipulation later on:

In [ ]:
dT   = df['dT[C]'].to_numpy()
Qdot = df['Qdot[W]'].to_numpy()

print("Temperature Numpy array: \n", dT)
print("Rate of heat loss Numpy array: \n", Qdot)

#### Loss Function

In linear regression, we are trying to find the values of $w$ and $b$ that fit the data as best we can. To this end, we need a way of assessing how far the current values are.

Our model (for D=1) has the form $f=wx+b$. We do not know what the optimal values for $w$ or $b$. For each data point, we can measure the mean of the square of the difference between the model's prediction $wx+b$ and the label:

$$\frac{1}{N} \sum_{i=1}^N(y_i - (wx_i+b))^2$$

If this is small, then the model is working well; conversely, if this is large, the model's prediction will be far from the labelled values at the known data points. We call this a loss function<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). We want to make the loss function as small as we can.

Let's denote the loss function as $l$:

$$l \equiv \frac{1}{N} \sum_{i=1}^N(y_i - (wx_i+b))^2$$

#### Gradient descent
We are interested in finding the values of $w$ and $b$ that minimise $l$. We can find the partial derivatives of the loss function with respect to the parameters $w$ and $b$. This will tell us how we might expect $l$ to change with a change in $w$ or $b$.

$$\frac{\partial l}{\partial w} = \frac{1}{N} \sum_{i=1}^N -2x_i(y_i - (wx_i+b))$$,
$$\frac{\partial l}{\partial b} = \frac{1}{N} \sum_{i=1}^N -2(y_i - (wx_i+b))$$.

With these partial derivatives, we can update $w$ and $b$ to drive down the loss: 

$$w \leftarrow w - \alpha \frac{\partial l}{\partial w}$$
$$b \leftarrow b - \alpha \frac{\partial l}{\partial b}$$

where $\alpha$ is the learning rate, and controls how quick the update is applied. Our updates for $w$ and $b$ can be implemented as follows:

In [ ]:
def update_wb( x, y, w, b, alpha ):
    """
    Updates w and b in an attempt to reduce loss.

    x: feature (in this case, x=dT, the temperature difference). 
    y: label (in this case, Qdot).
    w: The current value of w before its update. 
    b: The current value of b before its update.
    alpha: the learning rate.|
    """
    dl_dw = 0.0                                #initialse to zero
    dl_db = 0.0                                #initialse to zero

    #find the partial derivatives:
    for i in range(len(x)):                   #loop from i=0 to 23.
        dl_dw += -2.0*x[i]*(y[i]-(w*x[i]+b))
        dl_db += -2.0*(y[i]-(w*x[i]+b))

    dl_dw = dl_dw/float(len(x))
    dl_db = dl_db/float(len(x))

    w = w - alpha*dl_dw
    b = b - alpha*dl_db

    return w, b

We also need a way of evaluating the loss (or cost), $l$, for a given value of $w$ and $b$, as per the following function:

In [ ]:
def loss(x, y, w, b):
    l=0.0
    
    for i in range(len(x)):
        l += (y[i] - (w*x[i]+b))**2

    l = l/float(len(x))

    return l

With the two functions above defined, we can start to train our model. We can start with a guess of $w=0$ and $b=0$. These are the parameters of our model. We also need to choose a value of $\alpha$. For now, let's go for $\alpha=0.0001$ (more on this later). We will keep count of how many times the full dataset has been seen by the algorithm using the `epoch` integer.

<a id='training_cell'></a>

In [ ]:
w=0.0
b=0.0
alpha=0.0001

epoch = 0

w, b = update_wb(dT, Qdot, w, b, alpha)
epoch += 1
print("Epoch: ", epoch, "w: ", w, "b: ", b, "loss: ", loss(dT, Qdot, w, b))
        

OK, let's look at the results:

In [ ]:
plt.plot(dT, Qdot, marker = 'o', ls='')
plt.plot([0,25],[b,w*25+b])

Pretty terrible! We should try more epochs:

In [ ]:
for i in range(5): #lets try 5 more
    w, b = update_wb(dT, Qdot, w, b, alpha)
    epoch += 1
    print("Epoch: ", epoch, "w: ", w, "b: ", b, "loss: ", loss(dT, Qdot, w, b))

plt.plot(dT, Qdot, marker = 'o', ls='')
plt.plot([0,25],[b,w*25+b])

Better, but still a way to go!

In [ ]:
for i in range(200):  #let's try 200 more
    w, b = update_wb(dT, Qdot, w, b, alpha)
    epoch += 1
    print("Epoch: ", epoch, "w: ", w, "b: ", b, "loss: ", loss(dT, Qdot, w, b))

plt.plot(dT, Qdot, marker = 'o', ls='')
plt.plot([0,25],[b,w*25+b])
        

Looks good. 

## Things to try

- What happens if you go back and train with $\alpha=0.1$ or $1\times 10^{-5}$? Click [here](#training_cell) to return to the start of training. Change $\alpha$ and re-run the training cells to experiment.
- Change from gradient descent to Stochastic Gradient Descent; be careful to ensure the `epoch` integer corresponds to any $N$ samples being seen (even if the same sample is randomly chosen more than once within an epoch). (assessed)
- Change from MSE to MAE and compare performance. Is a different learning rate needed? (assessed)
- Prefer Matlab? Re-implement this linear regression code in matlab and play with it there.

## Linear regression with Scikit learn or Matlab

It is important to note that the above is a pedagogical example. In practical applications, you would not be well advised to implement your own linear regression. For one, this is reinventing the wheel. But perhaps even more importantly, the computational performance of our simple model is unlikely to be very good (i.e. it may take a long time to run on large datasets). In matlab, you can use the `fitlm` function to return a linear regression model. In Python, you can use Scikit-learn's `linear_model.LinearRegression()` model.

#### Footnotes

<a name="cite_note-1"></a>1. [^](#cite_ref-1) Formally, the loss function is the loss evaluated on a single data point (i.e. without the sum, or dividing by $N$) and the above is actually the "cost function". However, loss and cost are often used interchangeably, so with a minor abuse of notation to adopt the common practice in the literature, we'll call this loss.